# Data Modelling
This is a notebook to experiment with the data modelling of the sales quantity data.
This was done on a cloud instance so the file paths will be different if you are running this locally.
Note that the dataset is also propiertary so it will not be included in this repository.

# 1.Imports and Constants
We will be using the following libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

2023-06-11 13:17:43.465122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-11 13:17:44.357801: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-11 13:17:44.357915: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

In [2]:
WINDOW = 20
BATCH_SIZE = 2048
BUFFER = 100000

# Load data

The data is a csv extracted from an SQL database and cleaned. It contains the following columns:
- **date:** The date of the sale
- **item_code:** The code of the product sold
- **quantity:** The quantity of the product sold on that day

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# %pwd

In [4]:
# filepath = 'sales_quantity.csv' #for local imports
filepath = '/home/mariefloco/sales_quantity.csv' #colab
data = pd.read_csv(filepath,names=['date','item_code','quantity'],header = 0 )
data.head()

,date,item_code,quantity
0,2022-08-26,00001000,15
1,2022-08-26,00000500,14
2,2023-01-01,8991102380706,13
3,2023-01-01,8991102381017,13
4,2023-01-01,8886008101053,20


# Transform data
We need to change the data so that it has the following format for training:
- **Input:** [*The tokenized item code, day, month, day of the week, day of the year, {A sequence of 20 days of sales data for a particular product}*]
- **Output:** The quantity sold for that product in the following day
>**Note:**
    - *The tokenized item code is the index of the item code in the tokenizer's word index.*
    - *The day component of the date is the day of the month.*
    - *The month component of the date is the month of the year.*
    - *The day of the week is a number between 0 and 6, where 0 is Monday and 6 is Sunday.*
    - *The day of the year is a number between 1 and 365, where January 1st is 1 and December 31st is 365.*
    - *The sequence of 20 days of sales data is the window size we will use for training the model. The data will be normalized*


In [5]:
#extract date features from date column
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_year'] = data['date'].dt.dayofyear



We need to create a wide dataframe with each item code as a column and the quantity sold for each day as the values.

In [6]:
#stack dataframe based on item_code
item_sales = data.groupby(['item_code','date','year','month','day','day_of_week','day_of_year'])['quantity'].sum().unstack(level=0)
#turn each NaN value to 0
item_sales = item_sales.fillna(0)
item_sales.reset_index(inplace=True)
item_sales.head()

item_code,date,year,month,day,day_of_week,day_of_year,(90)NA18210500154(91)2403,(90)NA18211207820(91)2410,00000001,00000002,...,CL000448327,CL000450943,COS LT,COSLT-228,EC0102190002,EC0102191301,EC0103190002,EC0106190101,MP-2203,SLM0958266
0,2022-01-03,2022,1,3,0,3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-01-04,2022,1,4,1,4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-01-05,2022,1,5,2,5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-01-06,2022,1,6,3,6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-01-07,2022,1,7,4,7,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,2023-03-28,2023,3,28,1,87,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
447,2023-03-29,2023,3,29,2,88,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
448,2023-03-30,2023,3,30,3,89,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
449,2023-03-31,2023,3,31,4,90,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Since we will be passing the item code as a feature to the model, we need to tokenize it.
We use a helper function to create a tokenizer and fit it on the item codes.
This will be also be used to later decode the predictions of the model.

In [7]:
def create_tokenizer(item_code):
    """
    Create a tokenizer to tokenize item codes.

    Args:
        item_code (list or Series): List or Series containing item codes.

    Returns:
        tf.keras.preprocessing.text.Tokenizer: Tokenizer object fitted on item codes.
    """

    # Create a tokenizer with no filters and case-sensitive tokenization
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', lower=False)

    # Fit the tokenizer on the item codes
    tokenizer.fit_on_texts(item_code)

    return tokenizer

# Create a tokenizer using item codes from item_sales dataframe columns
tokenizer = create_tokenizer(item_sales.columns[6:].str.replace(' ', ''))

# Get the length of the tokenizer's word index
tokenizer_word_count = len(tokenizer.word_index)

print(f'Tokenizer has {tokenizer_word_count} tokens')
tokenizer.word_index

Tokenizer has 13917 tokens


{'(90)NA18210500154(91)2403': 1,
 '(90)NA18211207820(91)2410': 2,
 '00000001': 3,
 '00000002': 4,
 '00000003': 5,
 '00000008': 6,
 '00000010': 7,
 '00000011': 8,
 '00000012': 9,
 '00000013': 10,
 '00000014': 11,
 '00000015': 12,
 '00000016': 13,
 '00000017': 14,
 '00000019': 15,
 '00000020': 16,
 '00000021': 17,
 '00000022': 18,
 '00000023': 19,
 '00000024': 20,
 '00000025': 21,
 '00000026': 22,
 '00000027': 23,
 '00000030': 24,
 '00000031': 25,
 '00000032': 26,
 '00000034': 27,
 '00000035': 28,
 '00000036': 29,
 '00000037': 30,
 '00000038': 31,
 '00000039': 32,
 '00000040': 33,
 '00000041': 34,
 '00000042': 35,
 '00000044': 36,
 '00000045': 37,
 '00000046': 38,
 '00000047': 39,
 '00000048': 40,
 '00000049': 41,
 '00000050': 42,
 '00000051': 43,
 '00000052': 44,
 '00000054': 45,
 '00000057': 46,
 '00000058': 47,
 '00000059': 48,
 '00000060': 49,
 '00000061': 50,
 '00000062': 51,
 '00000063': 52,
 '00000064': 53,
 '00000065': 54,
 '00000067': 55,
 '00000069': 56,
 '00000070': 57,
 '0000

Now that we have the tokenizer, we can create the input and output data for the model.

We start with the input data.
Since we are using item codes as a feature, month, day, day of the week and day of the year, we need to create prefix features for each item code.

In [8]:
# Extract date values from item_sales dataframe columns
dates = np.array(item_sales[['month', 'day', 'day_of_week', 'day_of_year']])

# Perform cyclic encoding on the date values
dates_cyclic = np.sin(dates) + np.cos(dates)

prefix_features = []

# Iterate over each product in item_sales columns
for product in item_sales.columns[6:].str.replace(' ', ''):
    # Create prefix features for the product
    prefix_feature = np.array([
        [
            tokenizer.word_index[product],
            dates_cyclic[j][0],
            dates_cyclic[j][1],
            dates_cyclic[j][2],
            dates_cyclic[j][3]
        ]
        for j in range(WINDOW, len(item_sales))
    ], dtype=np.float64)
    
    prefix_features.append(prefix_feature)

# Get the total number of prefix features arrays and the shape of the first array
prefix_features_count = len(prefix_features)
prefix_features_shape = prefix_features[0].shape

print(f"A total of {prefix_features_count} numpy arrays with each one having shape {prefix_features_shape}")

print(prefix_features[0])

430

We not normalize the sales data in order to process later.

We create a list of tensorflow datasets, one for each item code. This will be used to create the windowed dataset using the Dataset API.

In [ ]:
#convert each item sales to a tensorflow dataset
sales_datasets = [tf.data.Dataset.from_tensor_slices(item_sales[column]) for column in
sales_datasets[0].element_spec

In [11]:
def window_dataset(token_time_ds, sales_ds, window_size):
    """
    Create a windowed dataset by combining token_time_ds and sales_ds.

    Args:
        token_time_ds (tf.data.Dataset): Dataset containing token and time information.
        sales_ds (tf.data.Dataset): Dataset containing sales information.
        window_size (int): Size of the window for creating sequences.

    Returns:
        tf.data.Dataset: Windowed dataset with input-output pairs.
    """

    # Create windows of size window_size+1, shifting by 1, and dropping any incomplete windows
    sales_ds = sales_ds.window(window_size+1, shift=1, drop_remainder=True)

    # Flatten the windows into individual datasets and combine them into a single dataset
    sales_ds = sales_ds.flat_map(lambda w: w.batch(window_size+1))

    # Concatenate token_time_ds and sales_ds tensors along the last axis
    windowed_tensors = tf.concat((list(token_time_ds), list(sales_ds)), axis=-1)

    # Create a new dataset from the concatenated tensors
    ds = tf.data.Dataset.from_tensor_slices(windowed_tensors)

    # Map each element of the dataset to input-output pairs
    ds = ds.map(lambda x: (x[:-1], x[-1]),num_parallel_calls=tf.data.AUTOTUNE)

    return ds


We create a windowed dataset for each item code. We use a window size of 20 days.
We clean up the memory by deleting the dataframes and series that are no longer needed. This is needed to avoid running out of memory.

In [13]:
#window the dataset in batches
ds = [window_dataset(prefix_features[i],sales_datasets[i], WINDOW) for i in range(len(sales_datasets))]
del data
del prefix_features
del sales_datasets
del item_sales

We split the dataset into training, validation, and test partitions.

In [15]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=False, shuffle_size=1000):
    """
    Splits a TensorFlow dataset into training, validation, and test partitions.

    Args:
        ds (tf.data.Dataset): The input dataset.
        ds_size (int): The total size of the input dataset.
        train_split (float, optional): The fraction of data to allocate for training. Defaults to 0.8.
        val_split (float, optional): The fraction of data to allocate for validation. Defaults to 0.1.
        test_split (float, optional): The fraction of data to allocate for testing. Defaults to 0.1.
        shuffle (bool, optional): Whether to shuffle the training dataset. Defaults to True.
        shuffle_size (int, optional): The buffer size used for shuffling. Defaults to 10000.

    Returns:
        tuple: A tuple containing the training, validation, and test partitions of the dataset.
    """
    assert (train_split + test_split + val_split) == 1, "The sum of train_split, val_split, and test_split must be 1."
    
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    if shuffle:
      # Specify seed to always have the same split distribution between runs
      train_ds = ds.take(train_size).shuffle(shuffle_size, seed=12)
    else:
      train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds


2023-06-11 13:17:49.746760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-11 13:17:49.758746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-11 13:17:49.760391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-11 13:17:49.762823: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild

KeyboardInterrupt: 

In [ ]:
#split the datasets to train, val, test
ds = [get_dataset_partitions_tf(items, 431, shuffle_size=BUFFER) for items in ds]

In [ ]:
#split into individual sets
train_set = [ds[i][0] for i in range(len(ds))]
val_set = [ds[i][1] for i in range(len(ds))]
test_set = [ds[i][2]for i in range(len(ds))]

In [ ]:
#turn into tensors
train_set = tf.data.experimental.from_list(train_set).flat_map(lambda x: x)
val_set = tf.data.experimental.from_list(val_set).flat_map(lambda x: x)
test_set = tf.data.experimental.from_list(test_set).flat_map(lambda x: x)

We need the length of each set for sampling later.
**Note:**
This process takes a long time. It's better once you have the length of each set to save it to a file and load it later.

In [ ]:
#find the length of each set using map
train_len = train_set.reduce(0, lambda x, _: x + 1).numpy()
val_len = val_set.reduce(0, lambda x, _: x + 1).numpy()
test_len = test_set.reduce(0, lambda x, _: x + 1).numpy()
# train_len = 4787448
# val_len = 598388
# test_len = 612348

print(f"Training set length: {train_len}")
print(f"Validation set length: {val_len}")
print(f"Test set length: {test_len}")


In [ ]:
#batch and prefetch the datasets
train_set = train_set.shuffle(BUFFER).batch(BATCH_SIZE).prefetch(1)
val_set = val_set.batch(BATCH_SIZE).prefetch(1)
test_set = test_set.batch(BATCH_SIZE).prefetch(1)

We save the datasets to disk so we can load them later. This is needed because the process of creating the datasets takes a long time.

In [ ]:
#save the datasets
tf.data.Dataset.save(train_set, 'train_set')
tf.data.Dataset.save(val_set, 'val_set')
tf.data.Dataset.save(test_set, 'test_set')

# Model Analysis

After preparing the dataset, we try to fine tune the learning rate of the algorithm.
We only use a sample of the training set to speed up the process.

In [ ]:
#create a list of models based on given learning rates and optimizers
def create_model():
    """
    Creates a list of models based on the learning rates and optimizers given.

    Args:
        learning_rate_array (list): A list of learning rates and optimizers to use for each model.

    Returns:
        list: A list of models.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                               strides=1,
                               activation="relu", padding="causal",
                               input_shape=[25, 1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    return model


In [ ]:
model = create_model()
train_sample = train_set.unbatch().shuffle(BUFFER).take(train_len//10).batch(BATCH_SIZE).prefetch(1)
val_sample = val_set.unbatch().shuffle(BUFFER).take(val_len//10).batch(BATCH_SIZE).prefetch(1)
train_sample

In [ ]:
#callback to tune the learning rate
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

optimizer = tf.keras.optimizers.SGD(momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mape"])

history = model.fit(train_sample, epochs=100, callbacks=[lr_schedule],validation_data = val_sample, verbose=2)


In [ ]:
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))

# Set the grid
plt.grid(True)

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')
plt.xlabel("Learning rate")
plt.ylabel("Loss")


# Model Training

We found that the best learning rate is 10e-5 (or 1e-4).

In [ ]:
#Reset the states generated by keras
tf.keras.backend.clear_session()

model = create_model()

In [ ]:
#set the learning rate
learning_rate = 1e-4
#set the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

#set the callback to stop the training if the validation loss doesn't improve
callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mape"])

history = model.fit(train_set, epochs=500, validation_data = val_set, verbose=2, callbacks=[callback])

In [ ]:
evaluation = model.evaluate(test_set)